In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import _LRScheduler
import torch.utils.data as data
from torch.utils.data import TensorDataset, DataLoader

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from tqdm.notebook import tqdm, trange
import matplotlib.pyplot as plt
import numpy as np

import copy
import random
import time
import os

In [32]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [33]:
def loadNpy(filename):
    with open(filename, "rb") as f: return np.load(f)

db = f"{os.getcwd()}/../data/slices/gaussian-grabber/size51/"
data_type = ""

In [35]:
X_train, X_val, X_test = loadNpy(db + f"X_train.npy"), loadNpy(db + f"X_val.npy"), loadNpy(db + f"X_test.npy")
y_train, y_val, y_test = loadNpy(db + f"c-y{data_type}_train.npy"), loadNpy(db + f"c-y{data_type}_val.npy"), loadNpy(db + f"c-y{data_type}_test.npy")

In [37]:
def npy_to_tensor(data, l=False):
    if not l:
        data = torch.from_numpy(data).float()
    else:
        data = torch.from_numpy(data).type(torch.LongTensor)
    return data

In [38]:
X_train, X_val, X_test = npy_to_tensor(X_train), npy_to_tensor(X_val), npy_to_tensor(X_test)
y_train, y_val, y_test = npy_to_tensor(y_train, 1), npy_to_tensor(y_val, 1), npy_to_tensor(y_test, 1)

In [39]:
batch_size = 100

In [40]:
train_dataset = TensorDataset(X_train, y_train)

val_dataset = TensorDataset(X_val, y_val)

test_dataset = TensorDataset(X_test, y_test)

# Data loader
train_iterator = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=False)

val_iterator = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)

test_iterator = DataLoader(dataset=test_dataset, batch_size=batch_size,shuffle=False)

In [36]:
y_train.shape

(840,)

In [46]:
input_size = 132651
h1_size = 100
h2_size = 100
h3_size = 100
h4_size = 100
h5_size = 100
num_classes = 2
num_epochs = 1000
batch_size = 64
learning_rate = 0.001

In [47]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, h1_size).to(device)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(h1_size, h2_size).to(device)
        self.relu2 = nn.ReLU()
        #self.drop1 = nn.Dropout(p=0.2)
        self.fc3 = nn.Linear(h2_size, h3_size).to(device)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(h3_size, h4_size).to(device)
        self.relu4 = nn.ReLU()
        #self.drop2 = nn.Dropout(p=0.2)
        self.fc5 = nn.Linear(h4_size, h5_size).to(device)
        self.relu5 = nn.ReLU()
        self.fc6 = nn.Linear(h5_size, num_classes).to(device)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        #out = self.drop1(out)
        out = self.fc3(out)
        out = self.relu3(out)
        out = self.fc4(out)
        out = self.relu4(out)
        #out = self.drop2(out)
        out = self.fc5(out)
        out = self.relu5(out)
        out = self.fc6(out)
        return out

In [48]:
def eval_nn(mod, data):
    preds = []
    actual = []
    
    shape = data.dataset.tensors[0].shape[1:]
    
    mod.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in data:
            images = images.to(device)
            labels = labels.to(device)
            outputs = mod(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            preds.append(predicted)
            actual.append(labels)

        print(f'Accuracy of the network on the {total} test images: {100 * correct / total} %')
    return preds, actual, correct/total

In [44]:
nn_model = NeuralNet(input_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(nn_model.parameters(), lr=learning_rate)

In [45]:
nn_epochs = {}

# Train the model
total_step = len(train_iterator)
for epoch in tqdm(range(num_epochs)):
    for i, (images, labels) in enumerate(train_iterator):  
        # Move tensors to the configured device
        #print(images.shape)
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        #print(images.shape)
        outputs = nn_model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print(i)

    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
           .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    nn_epochs[epoch] = {"loss": loss.item(), "train": eval_nn(nn_model, train_iterator), 
                 "val": eval_nn(nn_model, val_iterator), "test": eval_nn(nn_model, test_iterator)}
    nn_model.train()

  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch [1/1000], Step [9/9], Loss: 48.6786
Accuracy of the network on the 840 test images: 83.0952380952381 %
Accuracy of the network on the 280 test images: 86.07142857142857 %
Accuracy of the network on the 280 test images: 84.28571428571429 %
Epoch [2/1000], Step [9/9], Loss: 7.3497
Accuracy of the network on the 840 test images: 16.904761904761905 %
Accuracy of the network on the 280 test images: 13.928571428571429 %
Accuracy of the network on the 280 test images: 16.071428571428573 %
Epoch [3/1000], Step [9/9], Loss: 5.9076
Accuracy of the network on the 840 test images: 83.0952380952381 %
Accuracy of the network on the 280 test images: 86.07142857142857 %
Accuracy of the network on the 280 test images: 84.28571428571429 %
Epoch [4/1000], Step [9/9], Loss: 0.8114
Accuracy of the network on the 840 test images: 83.45238095238095 %
Accuracy of the network on the 280 test images: 86.07142857142857 %
Accuracy of the network on the 280 test images: 84.28571428571429 %
Epoch [5/1000], St

Epoch [35/1000], Step [9/9], Loss: 0.7877
Accuracy of the network on the 840 test images: 83.69047619047619 %
Accuracy of the network on the 280 test images: 86.07142857142857 %
Accuracy of the network on the 280 test images: 84.28571428571429 %
Epoch [36/1000], Step [9/9], Loss: 2.6612
Accuracy of the network on the 840 test images: 90.35714285714286 %
Accuracy of the network on the 280 test images: 85.35714285714286 %
Accuracy of the network on the 280 test images: 83.92857142857143 %
Epoch [37/1000], Step [9/9], Loss: 0.0378
Accuracy of the network on the 840 test images: 59.76190476190476 %
Accuracy of the network on the 280 test images: 37.5 %
Accuracy of the network on the 280 test images: 39.285714285714285 %
Epoch [38/1000], Step [9/9], Loss: 0.4746
Accuracy of the network on the 840 test images: 91.07142857142857 %
Accuracy of the network on the 280 test images: 67.5 %
Accuracy of the network on the 280 test images: 67.14285714285714 %
Epoch [39/1000], Step [9/9], Loss: 0.2601

KeyboardInterrupt: 